# JavaScript closures for beginners
https://stackoverflow.com/questions/111102/how-do-javascript-closures-work

### 例1

In [1]:
// 首先需要能理解下列代码，作为基础
function sayHello(name) {
  var text = 'Hello ' + name;
  var say = function() { console.log(text); }
  say();
}
sayHello('Joe');

Hello Joe


### Two one sentence summaries:
- A closure is one way of supporting first-class functions; it is an expression that can reference variables within its scope (when it was first declared), be assigned to a variable, be passed as an argument to a function, or be returned as a function result.
- Or, a closure is a stack frame which is allocated when a function starts its execution, and not freed after the function returns (as if a 'stack frame' were allocated on the heap rather than the stack!).

### 例2

In [2]:
function sayHello2(name) {
  var text = 'Hello ' + name; // Local variable
  var say = function() { console.log(text); }
  return say;
}
var say2 = sayHello2('Bob');
say2(); // logs "Hello Bob"

Hello Bob


这里JavaScript 和 C语言有关键性的不同。
- 在JavaScript里，函数引用变量（如例2中的say2）可以理解成
 - 既有一个指向函数的指针
 - 也包含一个隐藏的指向 Closure 的指针
- 在C语言里（以及其它大部分语言里），函数返回后，局部变量就不可以访问了，这是因为stack-frame被销毁了

例2之所以有闭包，是因为函数中还有一个匿名函数。在JavaScript里，如果你在另外一个函数内部使用 function 关键字，你就创建了一个闭包。

这里，sayHello2('Bob')已经返回了，但是我们还能访问sayHello2()中的局部变量 text

In [3]:
say2

function () { console.log(text); }

观察say2本身，这个匿名函数仍然引用 text，值应该是："Hello Bob"。魔法就在于，这个函数还有一个隐秘的对 closure 的引用。

**建议读者通过例子完全弄懂闭包再使用它，否则会遇到非常奇怪的错误！**

### 例3
该例子演示的是局部变量并没有新的拷贝，仅仅是个引用。

In [4]:
function say667() {
  // Local variable that ends up within closure
  var num = 42;
  var say = function() { console.log(num); }
  num++;
  return say;
}
var sayNumber = say667();
sayNumber(); // logs 43

43


### 例4
例子中三个全局变量引用相同的闭包

In [5]:
var gLogNumber, gIncreaseNumber, gSetNumber;
function setupSomeGlobals() {
  // Local variable that ends up within closure
  var num = 42;
  // Store some references to functions as global variables
  gLogNumber = function() { console.log(num); }
  gIncreaseNumber = function() { num++; }
  gSetNumber = function(x) { num = x; }
}

setupSomeGlobals();
gIncreaseNumber();
gLogNumber(); // 43

43


In [6]:
gSetNumber(5);
gLogNumber(); // 5

5


In [7]:
var oldLog = gLogNumber;

setupSomeGlobals();
gLogNumber(); // 42

42


In [8]:
oldLog() // 5

5


**注意**：setupSomeGlobals()第二次被调用的时候，一个新的闭包被创建了。在JavaScript里，每次当外部函数被调用时，内部函数都会被重新创建。

### 例5
这个例子可能会让很多人中招！**小心在循环中定义函数！**闭包中的局部变量可能和你想象的不一样。

In [9]:
function buildList(list) {
    var result = [];
    for (var i = 0; i < list.length; i++) {
        var item = 'item' + i;
        result.push( function() {console.log(item + ' ' + list[i])} );
    }
    return result;
}

function testList() {
    var fnlist = buildList([1,2,3]);
    // Using j only to help prevent confusion -- could use i.
    for (var j = 0; j < fnlist.length; j++) {
        fnlist[j]();
    }
}

testList() //logs "item2 undefined" 3 times

item2 undefined
item2 undefined
item2 undefined


返回的result里面有三个函数，并且共享相同的闭包。循环过后，
- item = 'item2'
- i = 3

### 例6
该例子说明了闭包包含所有的外部函数中局部变量，只要在函数退出之前。例子中alice变量在匿名函数之后定义的，也一样可以进入闭包，并被匿名函数引用。

In [10]:
function sayAlice() {
    var say = function() { console.log(alice); }
    // Local variable that ends up within closure
    var alice = 'Hello Alice';
    return say;
}
sayAlice()();// logs "Hello Alice"

Hello Alice


### 例7
例子显示每次调用都会创建一个独立的闭包，**并不是一个函数申明一本闭包。一次函数调用生成一个闭包**。

In [11]:
function newClosure(someNum, someRef) {
    // Local variables that end up within closure
    var num = someNum;
    var anArray = [1,2,3];
    var ref = someRef;
    return function(x) {
        num += x;
        anArray.push(num);
        console.log('num: ' + num +
            '; anArray: ' + anArray.toString() +
            '; ref.someVar: ' + ref.someVar + ';');
      }
}
obj = {someVar: 4};
fn1 = newClosure(4, obj);
fn1(1); // num: 5; anArray: 1,2,3,5; ref.someVar: 4;

num: 5; anArray: 1,2,3,5; ref.someVar: 4;


In [12]:
fn2 = newClosure(5, obj);
fn2(1); // num: 6; anArray: 1,2,3,6; ref.someVar: 4;

num: 6; anArray: 1,2,3,6; ref.someVar: 4;


In [13]:
obj.someVar++;
fn1(2); // num: 7; anArray: 1,2,3,5,7; ref.someVar: 5;

num: 7; anArray: 1,2,3,5,7; ref.someVar: 5;


In [14]:
fn2(2); // num: 8; anArray: 1,2,3,6,8; ref.someVar: 5;

num: 8; anArray: 1,2,3,6,8; ref.someVar: 5;


### 总结
- Whenever you use function inside another function, a closure is used.
- Whenever you use eval() inside a function, a closure is used. The text you eval can reference local variables of the function, and within eval you can even create new local variables by using eval('var foo = …')
- When you use new Function(…) (the Function constructor) inside a function, it does not create a closure. (The new function cannot reference the local variables of the outer function.)
- A closure in JavaScript is like keeping a copy of all the local variables, just as they were when a function exited.
- It is probably best to think that a closure is always created just an entry to a function, and the local variables are added to that closure.
- A new set of local variables is kept every time a function with a closure is called (given that the function contains a function declaration inside it, and a reference to that inside function is either returned or an external reference is kept for it in some way).
- Two functions might look like they have the same source text, but have completely different behaviour because of their 'hidden' closure. I don't think JavaScript code can actually find out if a function reference has a closure or not.
- If you are trying to do any dynamic source code modifications (for example: myFunction = Function(myFunction.toString().replace(/Hello/,'Hola'));), it won't work if myFunction is a closure (of course, you would never even think of doing source code string substitution at runtime, but...).
- It is possible to get function declarations within function declarations within functions — and you can get closures at more than one level.
- I think normally a closure is the term for both the function along with the variables that are captured. Note that I do not use that definition in this article!
- I suspect that closures in JavaScript differ from those normally found in functional languages.